# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: 
Date: 

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [2]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


## Helpers (use or modify)

In [8]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [11]:
SYMBOL = 'AAPL'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))

if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function': 'TIME_SERIES_DAILY_ADJUSTED','symbol': SYMBOL,'outputsize': 'compact','apikey': os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    msg = js.get('Note') or js.get('Information') or js.get('Error Message')
    keys = [k for k in js.keys() if 'Time Series' in k]
    if msg or not keys:
        print("No data returned. Reason:", msg or f"No 'Time Series' key. Keys: {list(js.keys())[:6]}")
        USE_ALPHA = False
    else:
        key = keys[0]
        df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','5. adjusted close':'adj_close'})
        if 'adj_close' not in df_api.columns and '4. close' in df_api.columns:
            df_api = df_api.rename(columns={'4. close':'adj_close'})
        df_api = df_api[['date','adj_close']]
        df_api['date'] = pd.to_datetime(df_api['date'])
        df_api['adj_close'] = pd.to_numeric(df_api['adj_close'], errors='coerce')
        source_name = 'alpha'

if not USE_ALPHA:
    import yfinance as yf
    tmp = yf.download(SYMBOL, period='3mo', interval='1d', auto_adjust=False, progress=False)
    if tmp is None or tmp.empty:
        tmp = yf.Ticker(SYMBOL).history(period='3mo', interval='1d', auto_adjust=False)
    tmp = tmp.reset_index()
    if 'Adj Close' in tmp.columns:
        df_api = tmp[['Date', 'Adj Close']]
    else:
        df_api = tmp[['Date', 'Close']]
    df_api.columns = ['date', 'adj_close']
    source_name = 'yfinance'


df_api = df_api.sort_values('date').reset_index(drop=True)
v_api = validate(df_api, ['date','adj_close'])
v_api


No data returned. Reason: Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints


{'missing': [], 'shape': (63, 2), 'na_total': 0}

In [12]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

Saved data\raw\api_source-yfinance_symbol-AAPL_20250820-162740.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [14]:
SCRAPE_URL = 'https://www.iban.com/currency-codes'  # TODO: replace with permitted page
headers = {'User-Agent':'AFE-Homework/1.0'}
try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)
except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

{'missing': [], 'shape': (269, 4), 'na_total': 0}

In [15]:
_ = save_csv(df_scrape, prefix='scrape', site='example', table='markets')

Saved data\raw\scrape_site-example_table-markets_20250820-163703.csv


## Documentation
- API Source:
 Yahoo Finance via `yfinance` (fallback)
 Ticker: `AAPL`
 Params: `period='3mo'`, `interval='1d'`, `auto_adjust=False`
 Notes: If `Adj Close` is missing, `Close`is used.
-if key present: Alpha Vantage
 URL:`https://www.alphavantage.co/query`
 Functions tried: `TIME_SERIES_DAILY_ADJUSTED` (may be premium) → fallback to `TIME_SERIES_DAILY`
 Params:`symbol=AAPL`, `outputsize=compact`, `apikey=<from .env>`
 Guard rails: If response has `Note/Information/Error Message` or no "Time Series" key, falls back to `yfinance`.

- Scrape Source: (URL/table description)
  URL: https://www.iban.com/currency-codes
- Table: Single table containing currency codes 
- Parsing: `BeautifulSoup` → collect all `<tr>`; for each row collect `th/td` text; first row is header, remainder is data.

- Validation Logic
  Helper: `validate(df, required)` reports:
  - missing required columns
  - `shape` (rows, cols)
  - total NA count and NA by column
  API DataFrame required columns:** `['date', 'adj_close']`
  - Type parsing: `date → datetime`, `adj_close → numeric (coerce)`
  - Sort ascending by `date`.
  Scrape DataFrame required columns:** list of all parsed columns from the header
  - Numeric coercion (if present): columns containing `['price','weight','float','cap','last','number']` are coerced with `str.replace('[,$]', '')` + `pd.to_numeric(..., errors='ignore')`

- Assumptions & risks: (rate limits, selector fragility, schema changes)
- Alpha Vantage:
  Rate limits and premium endpoints can return messages instead of data; code handles this and falls back to `yfinance`.
  Response schema may change (e.g., adjusted vs. close).
- yfinance / Yahoo Finance:
  Occasional outages or column differences (`Adj Close` vs `Close`).
  Market holidays may create gaps.
- Scraping (iban.com):
  HTML structure could change (selector fragility).
  
- Typing & Validation:
  Numeric coercion uses `errors='ignore'`; unexpected symbols may remain as text.
  Timezone is not adjusted; dates are treated as naive dates from the source.

- Confirm `.env` is not committed.